<a href="https://colab.research.google.com/github/caltunay/eng_lyrics_generator/blob/main/Indie_Song_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import sys
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
kaggle_oath = {"username":"","key":""}

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = kaggle_oath['username'] # username from the json file 
os.environ['KAGGLE_KEY'] = kaggle_oath['key'] # key from the json file

!kaggle datasets download -d mateibejan/multilingual-lyrics-for-genre-classification

 84% 86.0M/103M [00:01<00:00, 47.0MB/s]
100% 103M/103M [00:01<00:00, 96.3MB/s] 


In [ ]:
!unzip multilingual-lyrics-for-genre-classification.zip

Archive:  multilingual-lyrics-for-genre-classification.zip
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
songs = pd.read_csv('train.csv')

In [ ]:
songs.sample(5)

,Artist,Song,Genre,Language,Lyrics
267235,benji & fede,safari,Pop,it,siamo nati in posti complicati\nil tempo perso...
176439,archive,woman,Pop,en,I'm a woman\nYou're a man\nWas I put here for ...
104539,guilherme arantes,lendas,Pop,pt,Lendas\nVejo quantas lendas querem me prender\...
175505,carl perkins,boppin' the blues,Rock,en,Well all my friends are bopping the blues it m...
11882,cobra starship,anything for love,Pop,en,"Tell my friends I'm done, I'm not comin' back\..."


In [ ]:
songs.groupby('Language').size()

Language
af        77
ca       137
cs        17
cy       226
da        71
de       478
en    250197
es      3892
et        58
fi        54
fr       644
hr        97
hu        10
id       737
it       808
ko         1
lt         2
lv         2
nl       116
no        93
pl        24
pt     30102
ro      1184
ru         4
sk        98
sl        77
so       229
sq        15
sv        61
sw       304
tl       241
tr       116
vi         7
dtype: int64

In [ ]:
songs[songs['Language'] == 'tr']

,Artist,Song,Genre,Language,Lyrics
45676,queen,yeah,Rock,tr,Yeah
89147,timbaland,break ya back (feat. dev),Pop,tr,"Who that?\nBreak ya back, I'm tryna break ya b..."
100612,dima bilan,"tak, tak ne byvaet",Pop,tr,"Kazhdyy ras, kogda\nty podnimayesh glaza\nEto ..."
121419,t a t u,beliy plashik,Rock,tr,"Beliy Plashik\nYa risuyu chyernoy kraskoy,\nNa..."
121447,t a t u,malchik gey,Rock,tr,"Mal'chik Gey, Mal'chik Gey, Mal'chik Gey\nMal'..."
...,...,...,...,...,...
271084,allame,microfon utanir,Pop,tr,yuzumden dusen kac bin parca kelime deryasi\nl...
271085,allame,hari,Pop,tr,ye hari\npapagan otme bana\nbu dusman gotman\n...
271244,aspova,geber pislik,Pop,tr,"""belki baska bi' yolu vardir."" dedim\nbulamadi..."
271245,aspova,ne kaldi,Pop,tr,ne kaldi geriye benden la dondum deliye\ncok z...


In [ ]:
rand_inds = np.random.randint(0, len(songs)-1, 10)

for i in rand_inds:
    print(songs['Lyrics'][i])
    print('////////////////////')


In [ ]:
songs[songs['Language']=='en'].groupby('Genre').size()

Genre
Country         1890
Electronic      2005
Folk            8169
Hip-Hop         2238
Indie           7240
Jazz           13314
Metal          19133
Pop            86298
R&B             2765
Rock          107145
dtype: int64

In [ ]:
songs['lengths'] = songs['Lyrics'].apply(lambda x:len(str(x)))

In [ ]:
indie_lyrics = songs[(songs['Language']=='en')&(songs['Genre']=='Indie')&(songs['lengths']>35)].drop_duplicates()['Lyrics']

In [ ]:
indie_lyrics = indie_lyrics.reset_index(drop = True)

In [ ]:
indie_lyrics = indie_lyrics.str.lower()

In [ ]:
# vocab and char-ind dicts
sorted(set(' '.join(indie_lyrics)))

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '\x83',
 '\x85',
 '\x91',
 '\x92',
 '\x93',
 '\x9f',
 '\xa0',
 '¢',
 '¤',
 '¦',
 '©',
 'ª',
 '\xad',
 '®',
 '¯',
 '±',
 '´',
 '¹',
 '»',
 '½',
 '¿',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'ç',
 'é',
 'ê',
 'í',
 'ï',
 'ó',
 'ô',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

In [ ]:
search_for = ['\t',
 '\#',
 '\$',
 '\%',
 '\&',
  '\_',
 '\`',
 '\|'
#  "\'",
 '\(',
 '\(',
 '\)',
 '\*',
 '\+',
 '\/',
 '\=',
 '\>',
#  '\?',
 '\@',
 '\[',
 "\\",
 '\]',  
 '\^',       
 '\{',
 '\}',
 '\~',
 '\x83',
 '\x85',
 '\x91',
 '\x92',
 '\x93',
 '\x9f',
 '\xa0',
 '\¢',
 '\¤',
 '\¦',
 '\©',
 '\ª',
 '\xad',
 '\®',
 '\¯',
 '\±',
 '\´',
 '\¹',
 '\»',
 '\½',
 '\¿',
 '\à',
 '\á',
 '\â',
 '\ã',
 '\ä',
 '\ç',
 '\é',
 '\ê',
 '\í',
 '\ï',
 '\ó',
 '\ô',
 '\[',
 '\]']
#  '‘',
#  '’',
#  '“',
#  '”']

In [ ]:
'|'.join(search_for)

'\t|\\#|\\$|\\%|\\&|\\_|\\`|\\|\\(|\\(|\\)|\\*|\\+|\\/|\\=|\\>|\\@|\\[|\\|\\]|\\^|\\{|\\}|\\~|\x83|\x85|\x91|\x92|\x93|\x9f|\xa0|\\¢|\\¤|\\¦|\\©|\\ª|\xad|\\®|\\¯|\\±|\\´|\\¹|\\»|\\½|\\¿|\\à|\\á|\\â|\\ã|\\ä|\\ç|\\é|\\ê|\\í|\\ï|\\ó|\\ô|\\[|\\]'

In [ ]:
indie_lyrics = indie_lyrics[~indie_lyrics.str.contains('|'.join(search_for))]

In [ ]:
indie_lyrics

0       leave now while you can\n'cause growing old ju...
2       the needle's in hand,\nbut i cannot sew.\nmy h...
3       i catch every whisper\nsurrounding your head\n...
4       get off of work, come home,\npass out.\nmy lif...
5       you tried but you lost\nand your ship sank jus...
                              ...                        
6825    the sun burns down leaving god's bright stamp ...
6827    i thought they were kissing, but they were bit...
6829    i should have seen you were coming, i should h...
6830    mary, you would not be the woman that you are ...
6832    the day they found a cure for aids the day the...
Name: Lyrics, Length: 5502, dtype: object

In [ ]:
indie_lyrics = indie_lyrics.reset_index(drop = True)

In [ ]:
indie_lyrics_sampled = indie_lyrics.sample(int(np.ceil(len(indie_lyrics)/5)))

In [ ]:
indie_lyrics_sampled = indie_lyrics_sampled.reset_index(drop = True)
indie_lyrics_sampled

0       i wanna get stupid\ni gotta get limb\ni wanna ...
1       it's not too early for whiskey\nit's not too e...
2       i go on forever\nolder burns are overturned\nm...
3       with your coat of many colours\nand the flower...
4       we fell in love this summer\nby the spring tim...
                              ...                        
1096    we bury ourselves in our bottles\nbury ourselv...
1097    two island swans mated for life, \nand his fai...
1098    look who's talking to their shadow like one wh...
1099    \ntai toi she says\ntai toi she says\nso its l...
1100    i'm sick of has beens\ncoming home\ni'm coming...
Name: Lyrics, Length: 1101, dtype: object

In [ ]:
vocab = sorted(set(' '.join(indie_lyrics_sampled)))

In [ ]:
vocab

['\n',
 ' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '‘',
 '’',
 '“',
 '”',
 '…']

In [ ]:
# char-ind dicts

char_to_ind = {char:ind for ind, char in enumerate(vocab)}
ind_to_char = {ind:char for ind, char in enumerate(vocab)}

In [ ]:
max_len = 30
input_list = []
target_list = []

for song in indie_lyrics_sampled:
    for char_ind in range(0, len(song) - max_len):
        input_list.append(song[char_ind : char_ind + max_len])
        target_list.append(song[char_ind + max_len])


In [ ]:
print(len(input_list), len(target_list))

1016982 1016982


In [ ]:
x_arr = np.zeros((len(input_list), max_len, len(vocab) ), dtype = 'float32')
y_arr = np.zeros((len(target_list), len(vocab)), dtype = 'float32')


for ind_song, song in enumerate(input_list):
    for ind_char, char in enumerate(song):
        x_arr[ind_song, ind_char, char_to_ind[char]] = 1
    y_arr[ind_song , char_to_ind[target_list[ind_song]]] = 1

In [ ]:
# create baseline model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
import sys
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
model2 = Sequential()

model2.add(CuDNNLSTM(units = 512, input_shape = (max_len, len(vocab)), return_sequences = True))
model2.add(Dropout(.2))
model2.add(CuDNNLSTM(units = 512, return_sequences = True))
model2.add(Dropout(.2))
model2.add(CuDNNLSTM(units = 512))#, return_sequences = True))
model2.add(Dropout(.2))
# model2.add(CuDNNLSTM(units = 128, return_sequences = True))
# model2.add(Dropout(.2))
# model2.add(CuDNNLSTM(units = 128, return_sequences = True))
# model2.add(Dropout(.2))
# model2.add(CuDNNLSTM(units = 128))
# model2.add(Dropout(.2))

model2.add(Dense(len(vocab), activation = 'softmax'))

model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_6 (CuDNNLSTM)     (None, 30, 512)           1159168   
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 30, 512)           2101248   
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 52)               

In [ ]:
def sample(preds, temperature = 1.0):
    # helper to sample index from a prob array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature

    preds = np.exp(preds) / np.sum(np.exp(preds))
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end2(epoch, _):

    # function invoked at end of each epoch to print generated text
    print()
    print('----Generating text after epoch %d' % epoch+1)

    start_index = 0
    # text_starter = np.random.randint(0, len(indie_lyrics_sampled)-100)
    # text_ender = text_starter + 99 
    # lyric_ = text_[text_starter : text_ender]  # select random part of lyrics
    
    random_song_ind = np.random.randint(0, len(indie_lyrics_sampled)-1)
    random_song = indie_lyrics_sampled[random_song_ind]


    for temp in [.2, .5, 1]:
        print('Temperature at : ', temp)
        print('\n')
        generated = ''
        sentence = random_song[start_index : start_index + max_len]
        generated += sentence

        print('---Original lyrics (first 200 chars): \n', random_song[:200])
        print('\n---Generated with seed: \n"' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(300):
            x_pred = np.zeros((1, max_len, len(vocab)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_ind[char]] = 1

            preds = model2.predict(x_pred, verbose = 0)[0]
            next_index = sample(preds, temp)
            next_char = ind_to_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

        print()
        print('\n')

In [ ]:


# fit model
epoch_summary2 = LambdaCallback(on_epoch_end = on_epoch_end2)
earlyStopper = EarlyStopping(monitor = 'val_accuracy', patience = 5)

model_path = '/content/indie_song_generator(seed len 30).hdf5'

checkpoint_= ModelCheckpoint(model_path, monitor='val_accuracy', verbose=0, save_best_only=True)


model2.fit(x_arr, y_arr, epochs = 50, batch_size = 512, callbacks = [epoch_summary2, earlyStopper, checkpoint_], validation_split = .25, shuffle = True)

Epoch 1/50
1490/1490 [==============================] - 259s 173ms/step - loss: 2.4690 - accuracy: 0.3009 - val_loss: 1.5660 - val_accuracy: 0.5211

----Generating text after epoch 0
Temperature at :  0.2
---Original lyrics (first 200 chars):  i believe when we were younger we thought that we deserved the sun i met you when you were 18 i was pushing 21 the sentiment, the twists and turns begun living close to philadelphia we would go there 

---Generated with seed: "i believe when we were younger"
i believe when we were younger and i wanna be a little will be my heart to the dark of the street to the street the stranger the street to see it all the same the far the same
i wa


Temperature at :  0.5
---Original lyrics (first 200 chars):  i believe when we were younger we thought that we deserved the sun i met you when you were 18 i was pushing 21 the sentiment, the twists and turns begun living close to philadelphia we would go there 

---Generated with seed: "i believe when we were you

In [ ]:
model2.save('/content/drive/MyDrive/Colab Notebooks/indie_song_gen.h5')

In [ ]:
def generate_w_seed(sentence,temperature):
    sentence = sentence[0:max_len]
    # print(f'seed: {sentence}')
    print(f'temperature: {temperature}')
    generated = ''
    generated += sentence
    
    sys.stdout.write(generated)

    for i in range(500):
        x_pred = np.zeros((1, max_len, len(vocab)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_ind[char]] = 1

        preds = model2.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = ind_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    return

In [ ]:
for temp in [.2,.5,.8,1.0]:
    generate_w_seed("Stop making the eyes at me \nI'll stop making the eyes at you".lower(), temp)
    print()
    print('========================================================================\n')

temperature: 0.2
stop making the eyes at me 
i'm so happy to the defining in the morning and the sun burns the same all the times they have gone,
the band of the world to start the same mistakes
i'm gonna drive away the sun below me
i was the far and find another sleep and the stars that you could stay for an explosion
to the shores and the streets are fine
and i'm still trying to find the same all the things you have to be a promise more than i can see the same song anymore
and now your offering is really gone

and it's all on the way to yo


temperature: 0.5
stop making the eyes at me 
i'm so long and i don't want to be alone
i know i know i need you to believe in

i see you here with you and me. 
when you are there with you a bit through the constant state
to find out if all the times you have to play
then you've got a los to love me
and the sounds in the country and i know you can change your mind is done and i'm only human
and i'm not holding on anything i've got a lovely time

i 

In [ ]:
for temp in [.2,.5,.8,1.0]:
    generate_w_seed('love you more than craft beer '.lower(), temp)
    print()
    print('========================================================================\n')

temperature: 0.2
love you more than craft beer the same all for the sound of your shoes
there's no time to get out of this static
throw it all to the part of you

i know it's all about the money
but i'm so dreaming
you're so beautiful
you're beautiful today
you're a part of you

i want to see you through the wall
when you come around.
everything is gonna be alright

and i want to be the one to feel the same all the storm of the street
we are one and i'm still trying to find myself in the sky
i see the bloody that we come to the ground
i don'


temperature: 0.5
love you more than craft beer closed on the street
with the sun blazed by a glow
i'm standing in the matter or two show you are a savior
and all the times that you gave to me
then the world is yours
then there's something in the way
i see you nothing there are no in my heart

but when the ancient stars are different
i could be the love for us slowly and when i think of you lots sometimes
i love you and i love you like i do
and if

In [ ]:
for temp in [.2,.5,.8,1.0]:
    generate_w_seed('love you more than boozing up '.lower(), temp)
    print()
    print('========================================================================\n')

temperature: 0.2
love you more than boozing up and speaking in the sun
it was all that you could be the same now i'm so hard to stay

i want to be the greater you want to be the best friend i don't want to be alone like you
another one like you and i

i know it's all about the money
but i'm so under, i'm under, i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm under, i'm under
i'm


temperature: 0.5
love you more than boozing up for the shit i'm pretty bad now

there is no way out before
we are dicking our love like this
and i was wrong to remember
the words that let us be so are
and i will love like the world to show
when you come around.
so you'll see you to the rest of you and me and your heart
i can't let you know
i said that i will
i said that i was wrong with me by the light of the store to the best
white sun below me
